In [1]:
import boto3
import configparser
import psycopg2

In [2]:
parser = configparser.ConfigParser()
parser.read("pipeline.conf")
dbname = parser.get("redshift_creds", "database")
user = parser.get("redshift_creds", "username")
password = parser.get("redshift_creds", "password")
host = parser.get("redshift_creds", "host")
port = parser.get("redshift_creds", "port")

In [3]:
rs_conn = psycopg2.connect(
"dbname=" + dbname
+ " user=" + user
+ " password=" + password
+ " host=" + host
+ " port=" + port)

In [4]:
# load the account_id and iam_role from the
# conf files
parser = configparser.ConfigParser()
parser.read("pipeline.conf")
account_id = parser.get("aws_boto_credentials",
"account_id")
iam_role = parser.get("redshift_creds", "iam_role")
bucket_name = parser.get("aws_boto_credentials",
"bucket_name")

In [5]:
# run the COPY command to load the file into Redshift
file_path = ("s3://"
+ bucket_name
+ "/customers.csv")
role_string = ("arn:aws:iam::"
+ account_id
+ ":role/" + iam_role)
sql = "COPY public.customers" + " from %s " + " iam_role %s;"

In [6]:
# create a cursor object and execute the COPY
cur = rs_conn.cursor()
cur.execute(sql,(file_path, role_string))
# close the cursor and commit the transaction
cur.close()
rs_conn.commit()
# close the connection
rs_conn.close()